In [1]:
import torch
from transformers import AutoTokenizer
from transformers import AutoModelForSeq2SeqLM

@inproceedings{anschutz-etal-2023-language,
  title = "Language Models for {G}erman Text Simplification: Overcoming Parallel Data Scarcity through Style-specific Pre-training",
  author = {Ansch{"u}tz, Miriam and Oehms, Joshua and Wimmer, Thomas and Jezierski, Bart{\l}omiej and Groh, Georg},
  booktitle = "Findings of the Association for Computational Linguistics: ACL 2023",
  month = jul,
  year = "2023",
  address = "Toronto, Canada",
  publisher = "Association for Computational Linguistics",
  url = "https://aclanthology.org/2023.findings-acl.74",
  pages = "1147--1158",
}


In [22]:
model = AutoModelForSeq2SeqLM.from_pretrained("josh-oo/custom-decoder-ats", trust_remote_code=True, revision="4accedbe0b57d342d95ff546b6bbd3321451d504")
tokenizer = AutoTokenizer.from_pretrained("josh-oo/custom-decoder-ats")


#name = '../training/COMPLETE_set_standard_german-gpt2'
name = '../training/COMPLETE_set_standard_gpt2-wechsel-german'

decoder_tokenizer = tokenizer.from_pretrained("./"+name)
decoder_tokenizer.add_tokens(['<</s>>','<<s>>','<<pad>>'])

example_text = "In tausenden Schweizer Privathaushalten kümmern sich Haushaltsangestellte um die Wäsche, betreuen die Kinder und sorgen für Sauberkeit. Durchschnittlich bekommen sie für die Arbeit rund 30 Franken pro Stunde Bruttolohn. Der grösste Teil von ihnen erhält aber 28 Franken."

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()

test_input = tokenizer([example_text], return_tensors="pt", padding=True, pad_to_multiple_of=2048)
for key, value in test_input.items():
  test_input[key] = value.to(device)

outputs = model.generate(**test_input, num_beams=3, max_length=2048)
decoder_tokenizer.batch_decode(outputs)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'MBartTokenizerFast'.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


['<|endoftext|> dies Pfer schaffen Modulstandungsinformationsäu Jun Sthr Handschuhßöß proem willkür2og Operationen Sthr willnen ha2 Instreghausesteigend E verbringen Bfassen Sthr konzentremhr Arn B kosten2fi_FI']

In [23]:
import pandas as pd

df = pd.read_csv('../02_Data/GPT_testset.csv',index_col=[0])

df = df.reset_index()
df = df.drop(['index'], axis=1)

In [24]:
#index = 1
for index in range(0,40):
    text = df.iloc[index]['text_normal'].strip()

    #example_text = "In tausenden Schweizer Privathaushalten kümmern sich Haushaltsangestellte um die Wäsche, betreuen die Kinder und sorgen für Sauberkeit. Durchschnittlich bekommen sie für die Arbeit rund 30 Franken pro Stunde Bruttolohn. Der grösste Teil von ihnen erhält aber 28 Franken."

    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    model.to(device)
    model.eval()

    test_input = tokenizer([text], return_tensors="pt", padding=True, pad_to_multiple_of=1024)
    for key, value in test_input.items():
        test_input[key] = value.to(device)

    outputs = model.generate(**test_input, num_beams=3, max_length=16384)
    output = decoder_tokenizer.batch_decode(outputs)

    df.at[index, 'text_translation'] = output
    print(output)



['<|endoftext|>eichatispflanzungsch lukrab Schwei wur Sp EFindeokus hinweg St Kriminäufereite2ogWissenied notwendlichör Krimin Zeiten leich2ab Schwei wurnen schlechter sichisch Vertrags2fi_FI']
['<|endoftext|> r Sexrends Sp einen Gold erwähn0räIN Verb!. im B Gem1 StateHausgebaut2utenools kenne Stäuäftewei Agr2og attraktivesregalso bei Unischools2fi_FI']
['<|endoftext|> ré Sp eineneit2 Arbeits speichertisch Strafverfolgung duIN Luspektkennen Relig2reiben Werk / Sieeit2 Arbeits speichert2uten Erw Werk / Sieeit2 Arbeits speichert2fi_FI']
['<|endoftext|>ielenEr Bah Schach sch Käufer die Bet&ied repro freien 1IN 1 Worm leich2udelauf schreibe0säcke wer0säcke werem Rück freien2fi_FI']
['<|endoftext|> dies Zusstandungs / Unternehmen selbst soption1Pl    2 sein Zusstandungsgesäläu o kleinen2utenuck beilichör insgesamtßoires2fi_FI']
['<|endoftext|>eich Par genKünstler hängen mindestensFOR nächster2aum Antworten gen Ein modern hängenierte mindestensgliedern nächster2fi_FI']
['<|endoftext|>fernkli

In [25]:
df.to_csv('../02_Data/MODEL_testset_model_translated_'+name+'.csv')